In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [9]:
data = pd.read_csv("C:/Users/maila/OneDrive/Desktop/Manas/task3/Threats.csv")
data.head()

,Unnamed: 0,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,40000,40001,0.012996,tcp,-,FIN,16,18,1540,1644,...,1,6,0,0,0,5,7,0,Normal,0
1,40001,40002,0.004119,tcp,-,FIN,12,12,1064,2260,...,1,1,0,0,0,3,8,0,Normal,0
2,40002,40003,0.000988,udp,dns,CON,2,2,146,178,...,1,2,0,0,0,7,2,0,Normal,0
3,40003,40004,3.415787,tcp,ssh,FIN,230,238,24344,29556,...,1,1,0,0,0,4,1,0,Normal,0
4,40004,40005,0.193943,tcp,-,FIN,72,74,4238,63618,...,1,5,0,0,0,6,12,0,Normal,0


In [11]:
data.dtypes

Unnamed: 0             int64
id                     int64
dur                  float64
proto                 object
service               object
state                 object
spkts                  int64
dpkts                  int64
sbytes                 int64
dbytes                 int64
rate                 float64
sttl                   int64
dttl                   int64
sload                float64
dload                float64
sloss                  int64
dloss                  int64
sinpkt               float64
dinpkt               float64
sjit                 float64
djit                 float64
swin                   int64
stcpb                  int64
dtcpb                  int64
dwin                   int64
tcprtt               float64
synack               float64
ackdat               float64
smean                  int64
dmean                  int64
trans_depth            int64
response_body_len      int64
ct_srv_src             int64
ct_state_ttl           int64
ct_dst_ltm    

In [13]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, roc_auc_score, matthews_corrcoef, f1_score, recall_score

In [19]:
X = data.drop(columns = ['attack_cat'])
y = data['attack_cat']

In [23]:
has_nan = X.isnull().values.any()
print("Does X have any NaN values?", has_nan)

Does X have any NaN values? False


In [25]:
has_nan = y.isnull().values.any()
print("Does y have any NaN values?", has_nan)

Does y have any NaN values? False


In [29]:
non_numeric_cols = X.select_dtypes(include=['object']).columns
print("Non-numeric columns:", non_numeric_cols)

Non-numeric columns: Index(['proto', 'service', 'state'], dtype='object')


In [31]:
X = pd.get_dummies(X, columns=non_numeric_cols, drop_first=True)

In [33]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [35]:
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y,test_size = 0.3, stratify = y, random_state = 42)

In [39]:
dt = DecisionTreeClassifier(random_state = 42, class_weight = 'balanced')

In [41]:
dt.fit(x_train,y_train)

DecisionTreeClassifier(class_weight='balanced', random_state=42)

In [63]:
y_pred = dt.predict(x_test)
y_pred_proba = dt.predict_proba(x_test)[:,1]

In [65]:
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
                precision    recall  f1-score   support

      Analysis       0.25      0.24      0.25       600
      Backdoor       0.20      0.22      0.21       524
           DoS       0.35      0.37      0.36      3679
      Exploits       0.73      0.73      0.73     10018
       Fuzzers       0.89      0.87      0.88      5455
       Generic       0.99      0.98      0.99     12000
        Normal       1.00      1.00      1.00      4800
Reconnaissance       0.76      0.76      0.76      3148
     Shellcode       0.66      0.59      0.62       340
         Worms       0.61      0.59      0.60        39

      accuracy                           0.81     40603
     macro avg       0.64      0.63      0.64     40603
  weighted avg       0.81      0.81      0.81     40603



In [67]:
print("y_pred_proba shape:", y_pred_proba.shape)
print("y_test shape:", y_test.shape)

y_pred_proba shape: (40603,)
y_test shape: (40603,)


In [69]:
print("Sample predicted probabilities:", y_pred_proba[:5])

Sample predicted probabilities: [0. 0. 0. 0. 0.]


In [73]:
if len(y_pred_proba.shape) == 2:
    print("Probability sums (should all be ~1):", y_pred_proba.sum(axis=1))
else:
    print("y_pred_proba is not 2D. Shape:", y_pred_proba.shape)

y_pred_proba is not 2D. Shape: (40603,)


In [77]:
print("Number of feature names:", len(feature_names))
print("Number of importances:", len(importances))

Number of feature names: 45
Number of importances: 188


In [83]:
importances = dt.feature_importances_
feature_names = X_train.columns if hasattr(x_train, 'columns') else [f"Feature {i}" for i in range(x_train.shape[1])]
important_features = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})
print("Top 10 Important Features:")
print(important_features.sort_values(by="Importance", ascending=False).head(10))

Top 10 Important Features:
         Feature  Importance
5      Feature 5    0.136835
41    Feature 41    0.111111
173  Feature 173    0.103772
1      Feature 1    0.095638
0      Feature 0    0.080834
25    Feature 25    0.064595
176  Feature 176    0.047616
34    Feature 34    0.046720
26    Feature 26    0.036612
28    Feature 28    0.024942
